Python Examples

In [ ]:
# Read data from Excel and display using pandas
import pandas as pd

data = pd.read_csv('c:\\aaa\\titanic3.csv') 
data

In [ ]:
# Read data from SQL Server
import pyodbc
import sqlalchemy as sa
import pandas as pd
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas
import seaborn as sns
import matplotlib.pyplot as plt

# server parameters
server = 'BSQL' 
database = 'DBA' 
username = 'naya5' 
password = '9451P9Qt%&h!'
conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password

conn = pyodbc.connect(conn_string)
conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

query = ("select "
            "server, "
            "database_name, "
            "last_successful_log_restore, "
            "datediff(minute, last_successful_log_restore, getdate()) as minutes_since_last_restored "
        "from "
        "( "
        "select "
            "@@SERVERNAME as Server, "
            "b.database_name, "
            "max(a.restore_date) last_successful_log_restore "
        "from msdb..restorehistory a "
        "INNER JOIN msdb..backupset b ON a.backup_set_id = b.backup_set_id "
        "where  "
            "b.database_name IN ('Cetentities','cetgroups3','Common','Lms','Cetstore') "
        "group by b.database_name "
        ") a "
        )

# --- Print to terminal ---
#cursor = conn.cursor()
#cursor.execute(query)

#for i in cursor:
#    print(i)
# --- Print to terminal ---

# --- Print using pandas dataframe ---
engine = sa.create_engine(conn_url)
df = pd.read_sql(query, engine)
df.head()
# --- Print using pandas dataframe ---


In [ ]:
# Read data from MongoDB
from pymongo import MongoClient
import pandas as pd
import numpy as np
from tabulate import tabulate

# MongoDB connection string
myClient = MongoClient(
    host='172.17.30.23:27017',
    serverSelectionTimeoutMS=3000,
    username="sa",
    password="global11!"
)

# Select Database and Collection
myDB = myClient["test"]
myCollection = myDB["demo"]

# Query collection test.demo for number = '1000-10'
myQuery = {
        "balance" : {
            "$gte" : 20,
            "$lte" : 30
        }
    }

# Run query (using sort and limit)
myDoc = myCollection.find(myQuery).sort("balance", -1).limit(10)  # -1 descending

# Print using Pandas and tabulate
entries = list(myDoc)
df = pd.DataFrame(entries, columns = ['id', 'number', 'currency', 'balance'])
# print(tabulate(df, headers='keys', tablefmt='psql'))

# Print using pandas only (will work only inside a notebook)
df

# Print the results to the console
# for x in myDoc:
#    print(x)

In [ ]:
# Display Graphs
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(ncols=5, figsize=(30,5))
sns.violinplot(x="survived", y="age", hue="sex", data=data, ax=axs[0])
sns.pointplot(x="sibsp", y="survived", hue="sex", data=data, ax=axs[1])
sns.pointplot(x="parch", y="survived", hue="sex", data=data, ax=axs[2])
sns.pointplot(x="pclass", y="survived", hue="sex", data=data, ax=axs[3])
sns.violinplot(x="survived", y="fare", hue="sex", data=data, ax=axs[4])


In [ ]:
# Log Shipping bar graph from SQL Server
import pyodbc
import sqlalchemy as sa
import pandas as pd
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas
import seaborn as sns
import matplotlib.pyplot as plt

# server parameters
server = 'BSQL' 
database = 'DBA' 
username = 'naya5' 
password = '9451P9Qt%&h!'
conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password

conn = pyodbc.connect(conn_string)
conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

query = ("select "
            # "server, "
            "database_name, "
            "db_name, "
            # "last_successful_log_restore, "
            "datediff(minute, last_successful_log_restore, getdate()) as minutes_since_last_restored "
        "from "
        "( "
        "select "
            # "@@SERVERNAME as Server, "
            "b.database_name, "
            "b.database_name as db_name, "
            "max(a.restore_date) last_successful_log_restore "
        "from msdb..restorehistory a "
        "INNER JOIN msdb..backupset b ON a.backup_set_id = b.backup_set_id "
        "where  "
            "b.database_name IN ('Cetentities','cetgroups3','Common','Lms','Cetstore') "
        "group by b.database_name "
        ") a "
        )

# --- Print to terminal ---
#cursor = conn.cursor()
#cursor.execute(query)

#for i in cursor:
#    print(i)
# --- Print to terminal ---

# --- Print bar graph using pandas dataframe ---
engine = sa.create_engine(conn_url)
df = pd.read_sql(query, engine)

t=df.groupby(['database_name', 'db_name'])['minutes_since_last_restored'].sum().unstack('database_name').fillna(0)
t.plot(kind='bar', stacked=True)

plt.title('BSQL Log Shipping Sync Time')
plt.xlabel('Database')
plt.ylabel('Minutes Since Last Transaction Log was Restored')

# --- Print bar graph using pandas dataframe ---


In [ ]:
# Databases size bar graph from MongoDB
from pymongo import MongoClient
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt

# MongoDB connection string
myClient = MongoClient(
    host='172.17.30.23:27017',
    serverSelectionTimeoutMS=3000,
    username="sa",
    password="global11!"
)

# Set pandas dataframe with 2 columns
df = pd.DataFrame(columns=['database_name','sizeOnDisk'])

# Iterate over all databases and populate the dataframe
for dbname in enumerate(myClient.list_databases()):
    # print("database name is : ",dbname) # This is the original tupple
    database_name = dbname[1].get('name')
    size_on_disk  = dbname[1].get('sizeOnDisk')
    # print("database: ", database_name, ", Size: ", round(size_on_disk/1024/1024/1024,2), ' GB')
    
    # Populate the dataframe
    record = {'database': database_name, 'sizeOnDisk': size_on_disk }

    entry = pd.DataFrame.from_dict({
     "database_name": [database_name],
     "sizeOnDisk":  [round(size_on_disk/1024/1024/1024,2)]
    })
    df = pd.concat([df, entry], ignore_index=True)

# Display as graph
t=df.groupby(['database_name'])['sizeOnDisk'].sum().fillna(0)
t.plot(kind='bar', stacked=False)

plt.title('MongoDB Databases Size')
plt.xlabel('Database')
plt.ylabel('Size On Disk (GB)')

# df
